In [19]:
import pandas as pd
import io

df2 = train_df = pd.read_csv('drive/My Drive/Colab Notebooks/shuffledTraining4K.csv')
df2.head()


,id,qid1,qid2,question1,question2,is_duplicate
0,3300,6542,5314,Is there any possibility to increase height af...,How girls can increase their height after 18 y...,1
1,1366,2723,2724,I am a Portuguese national and I want to move ...,"I am 22 (working),I want to be cricketer becau...",0
2,981,1957,1958,Why is 'fahrenheit 451' perceived as dystopia?,Fahrenheit 451 (1953 book): What is Guy Montag...,0
3,1271,2534,2535,Who will probably win in the war between Pakis...,Who would win a *conventional* war between Ira...,1
4,1641,3267,3268,Do Trump supporters believe his claims about h...,Do Donald Trump supporters actually not care a...,1


In [0]:
import re

df2['question1_processed'] = df2['question1'].map(lambda x: re.sub('[,\.!?]|comments|answers|questions|question|answer|tags|comment|tag|post|community|ell', '', str(x)))
df2['question2_processed'] = df2['question2'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
df2['question1_processed'] = df2['question1_processed'].map(lambda x: x.lower())
df2['question2_processed'] = df2['question2_processed'].map(lambda x: x.lower())
sentences = df2['question1_processed'] + df2['question2_processed']


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

texts = [[word for word in word_tokenize(sentence)] for sentence in sentences]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
all_list = []
for text in texts:
    all_list += text
corpus = set(all_list)
print(corpus)

{'talk', 'whywhy', 'libertatians', 'animal', 'dies', 'upvotes', 'gin', 'obama', 'christ', 'thinkhow', 'ultimately', 'nice', 'blaspheming', 'snapchatif', 'desert', 'incorporate', 'materialistic', 'bees', 'ones', 'mentor', 'farwhat', 'blackfyre', 'people', 'marketingwhat', 'premier', 'judas', 'communismwhat', 'scumwhich', 'raisin', 'pi', 'developing', 'vicks', 'indiadaniel', 'stressis', 'childern', 'sodium', 'awesomewhat', 'gen', 'wyoming', 'actionwill', 'flyer', '30000rswhich', 'extra', 'shepherd-husky', 'w', 'sydneyin', 'rinse', 'ray', 'places', 'alberta', 'span', 'tricks', 'nokia', 'ola', 'testicle', 'ibm', 'maintaining', 'evernote', 'puducherry', 'info', 'muscles', 'elect', 'starting', 'az', 'inhabited', 'fats', 'notwhich', 'picturewhat', 'cornell', '&', 'anti-hacking/firewalling', 'quorawho', 'principle', 'henderson', 'improvement/clarification', 'wildlife', 'implications', 'extend', 'advanced', 'stayed', 'kitkat', 'laakmann', 'kakigardencom', 'area', 'iiiare', 'fifa', 'createdif', 

In [0]:
corpus_dict = dict(zip(corpus, range(len(corpus))))
# print(corpus_dict)


In [25]:
 def vector_rep(text, corpus_dict):
    vec = []
    for key in corpus_dict.keys():
        if key in text:
            vec.append((corpus_dict[key], text.count(key)))
        else:
            vec.append((corpus_dict[key], 0))

    vec = sorted(vec, key= lambda x: x[0])

    return vec

from math import sqrt
def similarity_with_2_sents(vec1, vec2):
    inner_product = 0
    square_length_vec1 = 0
    square_length_vec2 = 0
    for tup1, tup2 in zip(vec1, vec2):
        inner_product += tup1[1]*tup2[1]
        square_length_vec1 += tup1[1]**2
        square_length_vec2 += tup2[1]**2

    try:
      ans = (inner_product/sqrt(square_length_vec1*square_length_vec2))
    except:
      print("error")
      ans = 0

    return ans

vec1 = []
vec2 = []
correct = 0
wrong = 0
p = 0
isDuplicate = 0
notDup = 0
predictedDup = 0
predictedNotDup = 0
for x in range(0, 3000):
  vec1 = vector_rep(df2['question1_processed'][x], corpus_dict)
  vec2 = vector_rep(df2['question2_processed'][x], corpus_dict)
  cosine_sim = similarity_with_2_sents(vec1, vec2)
  if cosine_sim > 0.5:
    similarity = 1
  else:
    similarity = 0
  # dup = df2['is_duplicate']
  if df2['is_duplicate'][x] == 1:
    isDuplicate = isDuplicate + 1
  else:
    notDup = notDup + 1
  if similarity == 1:
    predictedDup = predictedDup + 1
  else:
    predictedNotDup = predictedNotDup + 1
  if similarity == df2['is_duplicate'][x] :
    correct = correct + 1
  else:
    if (p < 10):
      print("Sentence 1: ", df2['question1'][x])
      print("Sentence 2: ", df2['question2'][x])
      p = p + 1
      print(df2['is_duplicate'][x])
    wrong = wrong + 1

print("Correct: ", correct)
print("Wrong: ", wrong)
print("Actual Duplicate: ", isDuplicate)
print("Actual Not Duplicate: ", notDup)
print("Predicted Duplicate: ", predictedDup)
print("Predicted Not Duplicate: ", predictedNotDup)

Sentence 1:  I am a Portuguese national and I want to move to Norway. Is it possible?
Sentence 2:  I am 22 (working),I want to be cricketer because I enjoy cricket. Is it still possible for me to join cricket academy and get into national squad?
0
Sentence 1:  Why is 'fahrenheit 451'  perceived as dystopia?
Sentence 2:  Fahrenheit 451 (1953 book): What is Guy Montag's namesake?
0
Sentence 1:  What is the solution to this puzzle?
Sentence 2:  If you choose an answer to this question at random, what is the chance you will be correct? (A) 25% (B) 50% (C) 60% (D) 25%
0
Sentence 1:  What is the greatest mystery in the universe?
Sentence 2:  What is the greatest mystery of all time?
0
Sentence 1:  What is the function of nucleoplasm in a plant cell?
Sentence 2:  What is the function of centrosomes in plant cells?
0
Sentence 1:  What is the best way to introduce yourself in English?
Sentence 2:  How can you introduce yourself?
0
Sentence 1:  How do payroll companies make money?
Sentence 2:  H

In [0]:
print(len(texts))

404290
